# Notebook to compute requested anomaly for WP1 TiPACCs

## Modules

In [1]:
import numpy as np
import os
import copy
from datetime import datetime

# data
import netCDF4 as nc                             # read netcdf
import xarray as xr                              # write netcdf

## Define data

In [2]:
# tag
ctag='y2029.10y'

# case
cconfcase_ref='eORCA025.L121-OPM021'
cconfcase_ano='eORCA025.L121-OPM020'

# input file ISMIP6_eORCA025.L121-OPM021_y1999.10y_WP1_TiPACCs.nc
cfile_ref='ISMIP6_{}_{}_WP1_TiPACCS.nc'.format(cconfcase_ref,ctag)
cfile_ano='ISMIP6_{}_{}_WP1_TiPACCS.nc'.format(cconfcase_ano,ctag)

# output file
cf_out='ISMIP6_{}_{}_anomaly_WP1_TiPACCs.nc'.format(cconfcase_ano,ctag)

## Load data

In [3]:
# dataset in
ds_ref=xr.open_dataset(cfile_ref)
ds_ano=xr.open_dataset(cfile_ano)

# data set out
ds_out=xr.open_dataset(cfile_ano)

In [4]:
ds_out

<xarray.Dataset>
Dimensions:    (nbounds: 2, x: 761, y: 761, z: 30)
Coordinates:
  * x          (x) float64 -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * y          (y) float64 -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * z          (z) float64 -30.0 -90.0 -150.0 ... -1.65e+03 -1.71e+03 -1.77e+03
    longitude  (y, x) float64 ...
    latitude   (y, x) float64 ...
Dimensions without coordinates: nbounds
Data variables:
    z_bnds     (z, nbounds) float64 ...
    thetao     (z, y, x) float64 ...
    so         (z, y, x) float64 ...
    tob        (y, x) float64 ...
    sob        (y, x) float64 ...
    ficeshelf  (y, x) float64 ...
    depfli     (y, x) float64 ...
    deptho     (y, x) float64 ...
    crs        object ...
Attributes:
    source:         eORCA025.L121-OPM020 NEMO simulation
    model time:     10 years average starting 01/01/2029
    grid:           ISMIP6
    contact:        P. Mathiot (IGE)
    creation date:  2021-07-08 11:56:48.999833

## Compute anomaly

In [5]:
for cvar in ['thetao','so','tob','sob','ficeshelf']:
    dataout=ds_ano[cvar].to_masked_array() - ds_ref[cvar].to_masked_array()
    minval=np.min(dataout)
    maxval=np.max(dataout)
    clong_name=ds_ano[cvar].attrs['long_name']+' anomaly'
    ds_out[cvar].values = np.ma.masked_invalid(dataout).filled(fill_value=-9999.99)
    ds_out[cvar].attrs['valid_min']=minval
    ds_out[cvar].attrs['valid_max']=maxval
    ds_out[cvar].attrs['long_name']=clong_name
    ds_out=ds_out.rename({cvar : 'd_'+cvar})

## write file

In [6]:
ds_out

<xarray.Dataset>
Dimensions:      (nbounds: 2, x: 761, y: 761, z: 30)
Coordinates:
  * x            (x) float64 -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * y            (y) float64 -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * z            (z) float64 -30.0 -90.0 -150.0 ... -1.71e+03 -1.77e+03
    longitude    (y, x) float64 ...
    latitude     (y, x) float64 ...
Dimensions without coordinates: nbounds
Data variables:
    z_bnds       (z, nbounds) float64 -0.0 -60.0 -60.0 ... -1.74e+03 -1.8e+03
    d_thetao     (z, y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    d_so         (z, y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    d_tob        (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    d_sob        (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    d_ficeshelf  (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    depfli       (y, x) float64 ...
    deptho       (y, x) float64 ...
    crs          <U9 'epsg:3031'
Attributes:
    source:         eORCA025.L121-OPM020 NEMO simulation
    model time:     10 years average starting 01/01/2029
    grid:           ISMIP6
    contact:        P. Mathiot (IGE)
    creation date:  2021-07-08 11:56:48.999833

In [7]:
# global attributes
dgatt = {'description' : 'The above anomaly file is the differences between a tipped state and a reference state',
         'source' :'{} - {} NEMO simulations'.format(cconfcase_ano,cconfcase_ref),
         'model time' : '10 years average starting 01/01/1999',
         'grid'   : 'ISMIP6',
         'contact':'P. Mathiot (IGE)',
         'creation date':'{}'.format(datetime.now()),
        }

print('    write global att')
ds_out.attrs = dgatt

print('    write netcdf ...')
ds_out.to_netcdf(cf_out)
print('--------------------------------')

    write global att
    write netcdf ...
--------------------------------
